In [ ]:
%%capture
import ROOT
import glob
import math
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kError
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

In [ ]:
def drawQuantileProjection(objectName,rootDataFile,quantileOrder=0.5, axis="x"):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.QuantilesY(quantileOrder))
        else:
            quant = copy(histo.QuantilesX(quantileOrder))
        quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        quant.SetXTitle(histo.GetXaxis().GetTitle())
        quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("SAME L")
        legend.AddEntry(quant,runList[i])
    return quants,legend,canvas
    
def sliceAndFit(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60]):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        try:
            histo.FitSlicesY(myFunc, 0, -1, 0, "QNR")
            fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_1"))
        except:
            print("Could not perform fit.")
            return [],legend,canvas
        fit.SetYTitle("Gaus Fit Mean "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        fits.append(fit)
        legend.AddEntry(fit,runList[i])
        fit.Draw("SAME L")
    legend.Draw()
    return fits, legend, canvas

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetPalette(55)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
cols = list(ROOT.TColor.GetPalette())
def updateColors(histograms,palette):
    colors = []
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            color = palette[math.floor((i/(len(histograms)-1))*(len(palette)-1))]
        else:
            color = palette[0]
        #color = i
        colors.append(color)
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
    return colors

def updateRanges(histograms):
    maxRange = max([hist.GetMaximum() for hist in histograms])
    minRange = min([hist.GetMinimum() for hist in histograms])
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            hist.SetMaximum(maxRange)
            hist.SetMinimum(maxRange)


def getMedianHistogram(hists):
    from statistics import median

    if len(hists) == 0:
        raise ValueError("Histogram list is empty")

    # Determine if histograms are 1D or 2D
    is2D = isinstance(hists[0], ROOT.TH2)

    if len(hists) == 1:
        return hists[0]

    if is2D:
        # 2D histogram case
        nBinsX = hists[0].GetNbinsX()
        nBinsY = hists[0].GetNbinsY()
        xMin = hists[0].GetXaxis().GetXmin()
        xMax = hists[0].GetXaxis().GetXmax()
        yMin = hists[0].GetYaxis().GetXmin()
        yMax = hists[0].GetYaxis().GetXmax()
        medianHist = ROOT.TH2F("median_" + hists[0].GetName(), "Median " + hists[0].GetTitle(), nBinsX, xMin, xMax, nBinsY, yMin, yMax)
        for xBin in range(1, nBinsX + 1):
            for yBin in range(1, nBinsY + 1):
                vals = [h.GetBinContent(xBin, yBin) for h in hists]
                medianHist.SetBinContent(xBin, yBin, median(vals))
    else:
        # 1D histogram case
        nBins = hists[0].GetNbinsX()
        xMin = hists[0].GetXaxis().GetXmin()
        xMax = hists[0].GetXaxis().GetXmax()
        medianHist = ROOT.TH1F("median_" + hists[0].GetName(), "Median " + hists[0].GetTitle(), nBins, xMin, xMax)
        for xBin in range(1, nBins + 1):
            vals = [h.GetBinContent(xBin) for h in hists]
            medianHist.SetBinContent(xBin, median(vals))

    return medianHist

In [ ]:
def normalize_histogram(hist):
    if isinstance(hist[0], list):  # Check if the histogram is 2D
        # Convert 2D histogram bin contents to a probability distribution
        total = sum(sum(row) for row in hist)  # Sum all elements in the 2D histogram
        return [[h / total for h in row] for row in hist]
    else:
        # Convert 1D histogram bin contents to a probability distribution
        total = sum(hist)  # Sum all elements in the 1D histogram
        return [h / total for h in hist]



def calculate_kl_divergence(P, Q):
    # Flatten 2D histograms if necessary
    if isinstance(P[0], list):
        P = [item for sublist in P for item in sublist]
    if isinstance(Q[0], list):
        Q = [item for sublist in Q for item in sublist]

    # Calculate KL Divergence between two probability distributions P and Q
    return sum(p * math.log(p / q) for p, q in zip(P, Q) if p != 0 and q != 0)

def assign_quality(hists, medianHist):
    # Normalize the median histogram
    Q = normalize_histogram(medianHist)
    
    quality_scores = []
    for hist in hists:
        # Normalize the current histogram
        P = normalize_histogram(hist)
        
        # Calculate KL Divergence
        kl_div = calculate_kl_divergence(P, Q)
        quality_scores.append(kl_div)
        
    # Calculate the mean of the quality scores
    mean_score = sum(quality_scores) / len(quality_scores)

    # Calculate the variance of the quality scores
    variance = sum((score - mean_score) ** 2 for score in quality_scores) / len(quality_scores)

    # Calculate the standard deviation (sigma) of the quality scores
    sigma = math.sqrt(variance)

    quality = []
    for hist in hists:
        Q = normalize_histogram(hist)
        kl_div1 = calculate_kl_divergence(P, Q)
    
        # Assign quality based on KL divergence thresholds
        if abs(kl_div1) < sigma:
            quality.append("good")
        elif abs(kl_div1) < 2*sigma:
            quality.append("medium")
        else:
            quality.append("bad")
    
    return quality

def print_quality_scores(hists, medianHist, percent_diff=.20):
    # Normalize the median histogram
    Q = normalize_histogram(medianHist)
    kl_divergences = []

    # First pass: calculate KL divergences to determine the mean
    for hist in hists:
        P = normalize_histogram(hist)
        kl_div = calculate_kl_divergence(P, Q)
        kl_divergences.append(kl_div)

    # Calculate the mean of KL divergences
    mean_kl_div = sum(kl_divergences) / len(kl_divergences)

    # Calculate the variance of the quality scores
    variance = sum((score - mean_kl_div) ** 2 for score in kl_divergences) / len(kl_divergences)

    # Calculate the standard deviation (sigma) of the quality scores
    sigma = math.sqrt(variance)

    

    # Define thresholds based on mean and percentage difference
    threshold_good = mean_kl_div * (1 - percent_diff)
    threshold_bad = mean_kl_div * (1 + percent_diff)

    # Second pass: categorize histograms based on calculated thresholds
    for i, (hist, kl_div) in enumerate(zip(hists, kl_divergences)):
        if kl_div <= sigma:
            quality = "good"
        elif kl_div <= 2*sigma:
            quality = "medium"
        else:
            quality = "bad"
        
        print(f"Histogram {i+1}: Quality is {quality} (KL Divergence: {kl_div})")


In [ ]:
# Notebook variables
periodName = "LHC23o"
passName = "apass4"
runNumber = 123456
path = "/cave/alice-tpc-qc/data/2023/"

# Read the Root Files
fileList = glob.glob(path+"/"+periodName+"/"+passName+"/"+"*_QC.root")
fileList = [file for file in fileList if file[-13] != "_"]
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))

print(runList)

# TPC Async QC Report - myPeriod - myPass
### RCT Tables:
+ [2022](https://docs.google.com/spreadsheets/d/14vXFYVx3oVE8wgJKmIBhjvAt6NpGp7D6H4AmBM9E0Cw/edit#gid=0), [2023](https://docs.google.com/spreadsheets/d/1YBQLXWwwc3aC3B_PYVpFkTgEP0n6u1ovtYfiCOMWnTc/edit?pli=1#gid=0), [2023_PbPb](https://docs.google.com/spreadsheets/d/1vsl-pkLdUoNXZm6muPyMENWdDSKM7UDtQLcXc2B9QhE/edit#gid=492527515)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TPC-asyncQC" data-toc-modified-id="TPC-asyncQC">TPC asyncQC</a></span><ul class="toc-item"><li><span><a href="#General-notes" data-toc-modified-id="General-notes-1.1">General notes</a></span></li><li><span><a href="#Run-Table" data-toc-modified-id="Run-Table-1.2">Run Table</a></span></li></ul></li><li><span><a href="#Quality-Observer" data-toc-modified-id="Quality-Observer-2">Quality Observer</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-2.1">Clusters</a></span></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-2.2">Tracks</a></span></li></ul></li><li><span><a href="#Trendings" data-toc-modified-id="Trendings-3">Trendings</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-4">Plots</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-4.1">Clusters</a></span></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-4.2">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions-of-tracks" data-toc-modified-id="Geometrical-distributions-of-tracks-4.2.1">Geometrical distributions of tracks</a></span></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-4.2.2">Track properties</a></span></li></ul></li><li><span><a href="#PID" data-toc-modified-id="PID-4.3">PID</a></span><ul class="toc-item"><li><span><a href="#TPC-Gain-calibration" data-toc-modified-id="TPC-Gain-calibration-4.3.1">TPC Gain calibration</a></span></li><li><span><a href="#dEdx-vs-p" data-toc-modified-id="dEdx-vs-p-4.3.2">dEdx vs p</a></span></li></ul></li></ul></li></ul></div>

---
## Quality Observer

### Clusters

In [ ]:
# Use string formatting to dynamically set the output file name
outputFileName = f"{path}/{periodName}/{passName}/{periodName}_{passName}_QC.root"  # This will create 'LHC22e_QC.root'
# Create the output file with the dynamically set name
outputfile = ROOT.TFile(outputFileName, "RECREATE")

# Create the required directories and immediately write them
medianHists = outputfile.mkdir("medianHists")
medianHists.Write()  # Write the directory to ensure it's created

medianHistsTracksQC = medianHists.mkdir("TracksQC")
medianHistsTracksQC.Write()  # Write the subdirectory

trendings = outputfile.mkdir("trendings")
trendings.Write()  # Write the directory
trendingsTracksQC = trendings.mkdir("TracksQC")
trendingsTracksQC.Write()  # Write the subdirectory

kl_div = outputfile.mkdir("kl_div_trendings")
kl_div.Write()  # Write the directory
kl_divTracksQC = kl_div.mkdir("kl_divTracksQC")
kl_divTracksQC.Write()  # Write the subdirectory

# Ensure that the output file knows the current directory structure
outputfile.SaveSelf(True)

In [ ]:
qualityDF = pd.DataFrame({'runNumber': runList})
#these are not working right now "h2DRPhinClustersAside","h2DNClustersPhiAside","h2DRPhiqTotAside"
objects = ["hPhiAside","hPhiCside","hEta","h2DEtaPhi","hPt","hSign","hQOverPt","hNClustersAfterCuts","h2DNClustersPhiCside","h2DNClustersEta","h2DNClustersPt","hdEdxTotMIP_TPC","hdEdxTotMIP_IROC","hdEdxTotMIP_OROC1","hdEdxTotMIP_OROC2","hdEdxTotMIP_OROC3","hdEdxTotMIPVsSec_TPC","hdEdxTotMIPVsNcl_TPC","hdEdxTotMIPVsTgl_TPC","hdEdxTotMIPVsSnp_TPC","hdEdxTotVsP_TPC"]
trending = "mean"
error = "meanError"

for objectName in objects:
    if objectName == "hdEdxTotMIP_TPC;1":
        trending = "fit(gaus,Sq,N,40,60)"
    else:
        trending = "mean"

    # Draw trending
    [trend, canvas] = drawTrending(objectName, rootDataFile, names=runList, namesFromRunList=True, trend=trending, error=error, log="none", axis=1)

    # Check trending and quality, which returns the desired objects
    [qualities, canvas] = checkTrending(trend, canvas=canvas, thresholds={"GOOD": 1.5, "MEDIUM": 3, "BAD": 6})
    qualityDF[objectName] = qualities

    # Navigate to the desired directory
    outputfile.cd(trendingsTracksQC.GetPath())
    canvas.Write()

In [ ]:
#objects_for_median = ["hPhiAside","hPhiCside","hEta","h2DEtaPhi","hPt","hSign","hQOverPt","hNClustersAfterCuts","h2DNClustersPhiAside","h2DNClustersPhiCside","h2DNClustersEta","h2DNClustersPt","hdEdxTotMIP_TPC","hdEdxTotMIP_IROC","hdEdxTotMIP_OROC1","hdEdxTotMIP_OROC2","hdEdxTotMIP_OROC3","hdEdxTotMIPVsSec_TPC","hdEdxTotMIPVsNcl_TPC","hdEdxTotMIPVsTgl_TPC","hdEdxTotMIPVsSnp_TPC","h2DRPhinClustersAside","h2DRPhiqTotAside","hdEdxTotVsP_TPC"]
#these are not working right now "h2DRPhinClustersAside","h2DNClustersPhiAside","h2DRPhiqTotAside"
objects_for_median = ["hPhiAside","hPhiCside","hEta","h2DEtaPhi","hPt","hSign","hQOverPt","hNClustersAfterCuts","h2DNClustersPhiCside","h2DNClustersEta","h2DNClustersPt","hdEdxTotMIP_TPC","hdEdxTotMIP_IROC","hdEdxTotMIP_OROC1","hdEdxTotMIP_OROC2","hdEdxTotMIP_OROC3","hdEdxTotMIPVsSec_TPC","hdEdxTotMIPVsNcl_TPC","hdEdxTotMIPVsTgl_TPC","hdEdxTotMIPVsSnp_TPC","hdEdxTotVsP_TPC"]

for objects_for_median_name in objects_for_median:
    [hist, legend, canvas, pad1] = drawHistograms(objects_for_median_name, rootDataFile, normalize=True, legend=True, legendNames=runList, pads=False, drawOption="SAME L", xAxisRange=[-1.1, 1.1], grid=True)

    medianHist = getMedianHistogram(hist)

    if medianHist is None:
        continue  # Skip to the next iteration of the loop
        

    # Create a new canvas
    c = ROOT.TCanvas(objects_for_median_name, "Canvas for Histogram", 800, 600)
    
    # Draw the histogram on the new canvas
    medianHist.Draw()
    
    c.Update()

    # Navigate to the desired directory
    outputfile.cd(medianHistsTracksQC.GetPath())
    medianHist.Write()

    # Draw trending
    [trend, canvas] = drawTrending(objects_for_median_name, rootDataFile, names=runList, namesFromRunList=True, trend="kl_divergence", error=error, log="none", axis=1,meanHistogram=medianHist)

    # Check trending and quality, which returns the desired objects
    [qualities, canvas] = checkTrending(trend, canvas=canvas, thresholds={"GOOD": 1.5, "MEDIUM": 3, "BAD": 6})
    qualityDF[objects_for_median_name] = qualities
    
    # Navigate to the desired directory
    outputfile.cd(kl_divTracksQC.GetPath())
    canvas.Write()

In [ ]:
# Don't forget to close the ROOT file to save all changes
outputfile.Close()